## Prova antiga

1) Por que usar um sistema de gerenciamento de banco de dados e não um arquivo simples?

De fato, existem outras alternativas para um sistema de gerenciamento de bancos de dados, e isso porque nem sempre um banco de dados é necessário. Podem ser usados: um simples arquivo de texto, uma planilha Excel ou até um pedaço de papel. Mas,  precisamos considerar as seguintes situações: se o contexto envolve uma empresa, se há muitos usuários usando aquele arquivo de dados e se há muitos acessos, sem um gerenciamento de banco de dados provavelmente haverá fila para fazer acesso, lentidão, desorganização, mais complexidade para realizar certas operações, menor precisão e consistência, exposição de dados que deveriam estar sob sigilo e pouca proteção contra falhas. Dessa forma, usar um sistema de gerenciamento de dados que já foi concebido com todas essas situações sendo levadas em consideração é muito vantajoso, e é por isso que ele é tão usado em vendas e estoques, recursos humanos, dados científicos, informações geográficas, entre outros.

2) Qual a diferença entre WHERE e HAVING?

O HAVING é bem parecido com o WHERE, porém ele é aplicado apenas para grupos como um todo (isto é, para as colunas que também aparecem na clásula GROUP BY ou em uma função agregada - o que representa grupos), enquanto o WHERE é aplicado para linhas individuais de uma tabela. Importante resssaltar que uma query pode ter tanto o termo WHERE quanto o termo HAVING. Nesse caso, primeiro é aplicado o WHERE para as linhas individuais nas tabelas, que resultam em linhas que cumprem com as condições indicadas ali. Depois o HAVING será aplicado para as linhas no conjunto resultante, de tal forma que apenas grupos que atendam as condições estabelecidas no HAVING irão aparecer no query de saída.


3) Qual a diferença entre uma View e uma Temporary Table?

A VIEW é quase como se fosse uma apelido para o SELECT, funcionando apenas com uma operação de consulta (read-only), mas sendo dinâmica - de tal forma que se os dados originais mudarem, a view também mudará. Além disso, o objeto da VIEW é permanente, então caso seja interessante combinar algumas informações e disponibilizar dados diltrados para os usuários sem perdê-los, a VIEW é ideal. Por sua vez, a TEMPORARY TABLE é como se fosse um snapshot dos dados, funcionando não apenas com uma operação de consulta mas também com operações de edição, inserção (read-write). Outrossim, a temporary table é estática, de tal forma que faz apenas uma cópia dos dados originais sendo uma tabela independente. Por fim, o objeto da temporary table é temporário, não permanecendo no disco após o fim da conexão com o banco de dados, sendo principalmente usada como um passo intermediário de consulta.


In [3]:
import mysql.connector
from functools import partial


def get_connection_helper():

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print('Executando query:')
            cursor.execute(query, args)
            for result in cursor:
                print(result)

    connection = mysql.connector.connect(
        host='localhost',
        user='megadados',
        password='megadados',
        database='xpto',
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper()

In [4]:
#Fazendo um teste:

db('''
SELECT
    *
FROM 
    tipo
''')

db('''
SELECT
    *
FROM 
    ingredientetipo
''')

db('''
SELECT
    *
FROM 
    ingrediente
''')

db('''
SELECT
    *
FROM 
    ingredientepizza
''')

db('''
SELECT
    *
FROM 
    pizza
''')

Executando query:
('sem gluten',)
('sem lactose',)
('vegana',)
('vegetariana',)
Executando query:
('abobrinha', 'sem gluten')
('calabresa', 'sem gluten')
('massa sem gluten', 'sem gluten')
('mussarela', 'sem gluten')
('mussarela vegana', 'sem gluten')
('picanha', 'sem gluten')
('abobrinha', 'sem lactose')
('calabresa', 'sem lactose')
('massa sem gluten', 'sem lactose')
('mussarela vegana', 'sem lactose')
('picanha', 'sem lactose')
('abobrinha', 'vegana')
('massa sem gluten', 'vegana')
('mussarela vegana', 'vegana')
('abobrinha', 'vegetariana')
('massa', 'vegetariana')
('massa sem gluten', 'vegetariana')
('mussarela', 'vegetariana')
('mussarela vegana', 'vegetariana')
Executando query:
('abobrinha', 5.0)
('calabresa', 30.0)
('massa', 5.0)
('massa sem gluten', 7.0)
('mussarela', 10.0)
('mussarela vegana', 20.0)
('picanha', 40.0)
Executando query:
('calabresa', 'calabresa', 3.0)
('calabresa', 'massa', 1.0)
('incoerente', 'massa sem gluten', 1.0)
('incoerente', 'mussarela', 1.0)
('incoeren

1) O script “script_001.sql” contem a DDL para criação do banco de dados e também a DML para inserção de
alguns valores de exemplo, para ajudar vocês a responder as questões da prova. Se quiser, adicione
exemplos à vontade. Nas perguntas a seguir construa código SQL para cumprir o que se pede.

a) (1.0 pt) Liste as 3 pizzas que dão mais lucro. O lucro de uma pizza é o seu preço de venda menos o
custo total dos ingredientes nela utilizados (não se esqueça de que cada ingrediente da pizza tem uma
especificação de quantidade a ser utilizada).

In [48]:
db('''
SELECT 
    pizza, preco - SUM(preco_unitario*quantidade) as lucro
FROM 
    ingrediente
    INNER JOIN ingredientepizza USING (ingrediente)
    INNER JOIN pizza USING (pizza)
GROUP BY
    pizza
ORDER BY
    lucro DESC
LIMIT
    3
''')

Executando query:
('pão', 95.0)
('incoerente', 83.0)
('legumes', 78.0)


 (1.0 pt) Construa uma tabela temporária que liste todas as pizzas e a quantidade de ingredientes nela.

In [55]:
db('''
SELECT 
    pizza, ingrediente, COUNT(ingrediente) as qtd_ingrediente
FROM
    ingredientepizza
GROUP BY
    pizza
''')

Executando query:
('calabresa', 'calabresa', 2)
('incoerente', 'massa sem gluten', 4)
('legumes', 'abobrinha', 2)
('mussarela', 'massa', 2)
('pão', 'massa', 1)


In [56]:
db('DROP TEMPORARY TABLE IF EXISTS qtd_ingredientes_pizza')

db('''
CREATE TEMPORARY TABLE qtd_ingredientes_pizza
    SELECT 
        pizza, COUNT(ingrediente) as qtd_ingrediente
    FROM
        ingredientepizza
    GROUP BY
        pizza
''')

db('''
SELECT
    *
FROM qtd_ingredientes_pizza

''')

Executando query:
Executando query:
Executando query:
('calabresa', 2)
('incoerente', 4)
('legumes', 2)
('mussarela', 2)
('pão', 1)


(1.0 pt) Construa uma tabela temporária que liste todas as pizzas a quantidade de ingredientes
veganos nela (ou seja, onde o tipo é “vegana”)

In [57]:
db('''
SELECT 
    pizza, COUNT(ingrediente) as qtd_ingrediente
FROM
    ingredientepizza
    INNER JOIN ingredientetipo USING(ingrediente)
WHERE 
    tipo = 'vegana'
GROUP BY
    pizza
''')

Executando query:
('legumes', 2)
('incoerente', 2)


In [58]:
db('DROP TEMPORARY TABLE IF EXISTS qtd_ingredientes_pizza_veganos')

db('''
CREATE TEMPORARY TABLE qtd_ingredientes_pizza_veganos
    SELECT 
        pizza, COUNT(ingrediente) as qtd_ingrediente
    FROM
        ingredientepizza
        INNER JOIN ingredientetipo USING(ingrediente)
    WHERE 
        tipo = 'vegana'
    GROUP BY
        pizza
''')


db('''
SELECT
    *
FROM qtd_ingredientes_pizza_veganos

''')

Executando query:
Executando query:
Executando query:
('legumes', 2)
('incoerente', 2)


 (1.0 pt) Usando as tabelas temporárias dos itens (b) e (c) – mesmo que você não os tenha feito – liste
as pizzas veganas da pizzaria. Uma pizza é vegana se todos os seus ingredientes são veganos.


In [59]:
db('''
SELECT
    pizza
FROM
    qtd_ingredientes_pizza
    INNER JOIN qtd_ingredientes_pizza_veganos USING (pizza)
WHERE
    qtd_ingredientes_pizza.qtd_ingrediente = qtd_ingredientes_pizza_veganos.qtd_ingrediente 
''')

Executando query:
('legumes',)


(1.0 pt) Liste os ingredientes e seus tipos, um ingrediente por linha (Dica: use a função
GROUP_CONCAT). O resultado deve ser da seguinte forma:

In [63]:
db('''
SELECT
    ingrediente, CONCAT(GROUP_CONCAT(tipo SEPARATOR ", ")) as lista_tipos
FROM
    ingredientetipo
GROUP BY
    ingrediente
''')

Executando query:
('abobrinha', 'sem gluten, sem lactose, vegana, vegetariana')
('calabresa', 'sem gluten, sem lactose')
('massa', 'vegetariana')
('massa sem gluten', 'sem gluten, sem lactose, vegana, vegetariana')
('mussarela', 'sem gluten, vegetariana')
('mussarela vegana', 'sem gluten, sem lactose, vegana, vegetariana')
('picanha', 'sem gluten, sem lactose')


In [64]:
connection.close()